<a href="https://colab.research.google.com/github/besmets/RSE_course/blob/main/RSE_Lecture_02_RS_of_ash_and_gas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REMOTE SENSING OF THE ENVIRONMENT - LECTURE 2
# Introduction to Google Earth Engine and remote sensing of ash and aerosols
(c) Vrije Universiteit Brussel, Prof. Dr. Benoît SMETS - 2023-2024

------------
<br>

### Objectives of this tutorial:
1. TOOL: Introduction of Google Earth Engine with Python  
2. PROCESSING: Discover geostationary imagery (GOES-17) and multispectral imagery dedicated to gas and aerosols (Sentinel-5P/TROPOMI)  

### PREREQUISITES
- Computer with internet connection and web browser.  
- Google GMAIL account.  
- A direct access to Google Earth Engine, via the [creation of a cloud project](https://console.cloud.google.com/earth-engine/welcome?pli=1&inv=1&invt=Abp4yw).

### TO START THE EXERCISE

To start using this notebook, first copy it on your drive.
--> On the upper toolbar, click on "Save a Copy in Drive".

<br>
To make this tutorial work, you have to systematically run all cells containing code lines, from the first to the last one (Type SHIFT+ENTER to run a selected cell).  

<br>

------------

## PART 1 – Introduction to the Earth Engine Python API

*(This tutorial is strongly inspired by the documentation provided by Prof. Qiusheng Wu on https://courses.geemap.org/ and his [book](https://book.geemap.org/))*

### 1.1.  Let's first import the necessary packages:

In [ ]:
# install GEEMAP in the COLAB environment
!pip install geemap

# import the libraries
import ee
import geemap

### 1.2. Connect to Google Earth Engine (GEE)

The following code (here below) will create an interactive map in the notebook. To do so, we first need to authenticate to GEE.   

Authorise the access by specifying the name of you GEE project *(see line 3 of the code here below)*.   

Click SHIFT + Enter to run the cell here below !   

--> You should see the interactive map appearing.


In [ ]:
# Authenticate
ee.Authenticate()
ee.Initialize(project='your-project-name')   # To adapt with the name of your cloud project

# Display the interactive map
Map = geemap.Map(center=[50.8,4.4], zoom=6)
Map

### 1.3.  Open satellite imagery

To check which collections of satellite images are available in GEE, you can visit the dataset page of GEE: https://developers.google.com/earth-engine/datasets.  

For this tutorial, we are going to upload Sentinel-2 imagery over Belgium.

Information to upload Sentinel-2 data is [available here](https://developers.google.com/earth-engine/datasets/catalog/sentinel-2).   

There are 2 version of the collection:   
- Surface reflectance *(corrected from the atmosphere)*.  
- Top-of-Atmosphere *(TOA) Reflectance (no atmospheric correction)*.   

Let's use the surface reflectance version!

In [ ]:
# Create a variable "collection", calling the collection needed
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Let's filter this collection to only select July 2023
s2_collection_202307 = s2_collection.filterDate('2023-07-01', '2023-07-31')

# Now, we are only interest by images covering Brussels. Let's filter again!
roi = ee.Geometry.BBox(4.05, 50.45, 5.24, 51.2)  # Coordinates = (west, south, east, north)
s2_collection_Bel_202307 = s2_collection_202307.filterBounds(roi)

Note that you can simplify the selection of the collection with something like this:   

```python  
# Filtering parameters
start_date = '2023-07-01'
end_date = '2023-07-31'
roi = ee.Geometry.BBox(4.05, 50.45, 5.24, 51.2)

#Collection
dataset = s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
.filterDate(start_date, end_date) \
.filterBounds(roi)
```

Now, let's see what we have for the month of July 2023!

In [ ]:
# Get the collection size
print(s2_collection_Bel_202307.size().getInfo(), 'images')

48 images


In [ ]:
# Get the first image and display the metadata
first_image = s2_collection_Bel_202307.first()
geemap.image_props(first_image).getInfo()

In [ ]:
# Display the first image in the interactive map
Map.addLayer(first_image, {}, "First image")
Map.centerObject(first_image, 6)
Map

Ok... That's not convincing. We cannot clearly see the image and we only have a fraction of it. Let's select another image and improve the display!

In [ ]:
# Look at the content of the filtered collection
display(s2_collection_Bel_202307)

In [ ]:
# Select one image in the list
collection_list = s2_collection_Bel_202307.toList(s2_collection_Bel_202307.size())
display(collection_list.get(5))

In [ ]:
# Select one image in the list and display it on the interactive map
selected_image = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20230704T104621_20230704T104624_T31UES')

# Tune the visualisation parameters
visParams = {
  "bands": ['B4', 'B3', 'B2'],
  "min": -150,
  "max": 4000,
}

# Display on the map
Map.addLayer(selected_image, visParams, 'selected image')
Map.centerObject(selected_image)
Map

Well, the display is better, but we still have lots of clouds. Let's filter the collection to only keep images with (almost) no clouds!

In [ ]:
S2_coll_cloudfree = s2_collection_Bel_202307.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))   # lt() filter stands for "less than" something

display(S2_coll_cloudfree)

In [ ]:
selected_image = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20230711T103631_20230711T104102_T31UFR')

# Tune the visualisation parameters
visParams = {
  "bands": ['B4', 'B3', 'B2'],
  "min": -150,
  "max": 4000,
}

# Display on the map
Map.addLayer(selected_image, visParams, 'selected image')
Map.centerObject(selected_image)
Map

Great! It works and we can see the ground surface. Know, let's play with the spectral bands and the visualisation parameters!   

Check https://developers.google.com/earth-engine/guides/image_visualization for more visualisation parameters!

In [ ]:
# EXERCISE:
# Select the bands to create the composite image! --> Visit https://developers.google.com/earth-engine/datasets
selected_bands = [, , ]
min =
max =

new_visParams = {
  "bands": selected_bands,
  "min": min,
  "max": max,
}

# Display on the map
Map.addLayer(selected_image, new_visParams, 'selected image')
Map.centerObject(selected_image)
Map

Now, you can deal with collections of images, their filtering, and the display of images in the interactive map.

**Note:** You can explore basic functions to display information on images by looking at https://developers.google.com/earth-engine/guides/image_info.  

-------------------

## PART 2 – Remote Sensing of Ash using GEE

In this exercise, we will use GOES-17 imagery to look at ash emissions related to the Hunga Tunga 2022 eruption.

###2.1 Loading the GOES-17 collection

In [ ]:
# Load the collection and filter for a given date range

collection_name = 'NOAA/GOES/17/MCMIPF'
start_datetime = '2022-01-15T04:00:00'
end_datetime = '2022-01-15T05:35:00'

goes17_coll = ee.ImageCollection(collection_name).filterDate(start_datetime, end_datetime)

# Display information on the collection
display(goes17_coll)

Look at the content!  

- *How many images are available?*  
- *How many spectral bands have the images?*  
- *What is the pixel depth of the spectral bands?*  

<br>

Not all spectral bands are useful. Let's select only some of them!

In [ ]:
# Create a collection of images with selected spectral bands

selected_bands = ['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C11', 'CMI_C14', 'CMI_C15']

goes17_coll_lite = ee.ImageCollection(collection_name).filterDate(start_datetime, end_datetime).select(selected_bands)

# Display information on the collection
display(goes17_coll_lite)

Look at the content!  

- *How many spectral bands have the images?*  

### 2.2.  Create a green band and crop the images

The GOES-17 imagery does not have a green band, which prevents the creation of a true-color composite image. Fortunately, [Bah et al. (2018)](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2018EA000379) proposed a way to used use the blue, red and NIR bands to artificially create a green-like band. Let's apply this to display true-color images!  

Let's also crop the collection to retain only our region of interest over Hunga Tunga!  

In [ ]:
# Define band aliases.

BLUE = 'CMI_C01'
RED = 'CMI_C02'
VEGGIE = 'CMI_C03'

# Define the functions of the Bah et al. (2018) procedure

def applyScaleAndOffset(img):
  def getFactorImg(factorNames):
    factorList = img.toDictionary().select(factorNames).values()
    return ee.Image.constant(factorList)

  scaleImg = getFactorImg(['CMI_C.._scale'])
  offsetImg = getFactorImg(['CMI_C.._offset'])
  scaled = img.select('CMI_C..').multiply(scaleImg).add(offsetImg)
  return img.addBands(srcImg=scaled, overwrite=True)

def addGreenBand(img):
  green = img.expression('GREEN = 0.45 * red + 0.10 * nir + 0.45 * blue', {
    'blue': img.select(BLUE),
    'red': img.select(RED),
    'nir': img.select(VEGGIE)})
  return img.addBands(green)

# Crop the images to a specific ROI

def cropImage(img):
  roi = ee.Geometry.BBox(170, -29.75, 200, -9.7)
  img.clip(roi)

### 2.3.  Display a GOES image over the Hunga Tunga 2022 eruption

In [ ]:
# Modify the image to add the green band

selected_image = ee.Image('NOAA/GOES/17/MCMIPF/2022015053032000000')

image = addGreenBand(applyScaleAndOffset(selected_image))

display(image)

# Define the visualisation parameters

GOES_MIN = 0.0
GOES_MAX = 0.7;  # Alternatively 1.0 or 1.3.
GAMMA = 1.3

goesRgbViz = {
  'bands': [RED, 'GREEN', BLUE],
  'min': GOES_MIN,
  'max': GOES_MAX,
  'gamma': GAMMA,
  'opacity': 1
}

roi = ee.Geometry.BBox(170, -29.7, 200, -9.7)

# Display the result
Map.clear()
Map = geemap.Map()
Map.addLayer(image.clip(roi), goesRgbViz, 'GOES-17 RGB composite')
Map.centerObject(roi)
Map

Change the name of the image to see other images of the collection!

###2.4.  Export the images for manipulation outside GEE

GEEMAP has a timeseries function allowing us to directly create a list of images and move from one image to another using a slider.  

However, such function is not used here, as we would like to make a more advanced time-lapse animation.  

So, let's first export the selected images, together with the created green band!

In [ ]:
# Check the number of images in the filtered collection, and their name
goes17_coll_lite.aggregate_array('system:index').getInfo()

In [ ]:
# Apply the Bah et al. (2018) procedure to the collection and crop images

def update_collection(img):
  roi = ee.Geometry.BBox(178.88, -24.6, 189.32, -16.25)
  return addGreenBand(applyScaleAndOffset(img.clip(roi)))

images = goes17_coll.map(update_collection)

band_selection = ['CMI_C01', 'CMI_C02', 'GREEN', 'CMI_C03', 'CMI_C11', 'CMI_C14', 'CMI_C15']
images_selected_bands = images.select(band_selection)

display(images)
display(images_selected_bands)

print(' ')
print('Image collection list:')
images.aggregate_array('system:index').getInfo()

Look at the content for both collections!  

- *How many images are available?*  

- *How many spectral bands have the images?*  

### 2.5.  Export the images for manipulation outside GEE

A GEEMAP function exists to export images, either on a local disk or on Google Drive:  

```python
# Export image to local drive
geemap.ee_export_image()

# Export image to Google Drive
geemap.ee_export_image_to_drive()

# Export collection to local drive
geemap.ee_export_image_collection()

# Export collection to Google Drive
geemap.ee_export_image_collection_to_drive()
```

Unfortunately, the export option for a local drive limits the size of an exported image to 32 MB.  

We will, consequently, use Google Drive...

In [ ]:
export_folder = '__RSE_colab_data' # just direct folders (no subfolders)
pixel_size = 1000;                 # in metres
chosen_crs = 'EPSG:4326'           # Geographic or Projected Coordinate System
fileformat = 'GeoTIFF'

geemap.ee_export_image_collection_to_drive(images_selected_bands, folder=export_folder, scale=pixel_size, crs=chosen_crs)

Export progress can be visualised in the task window of GEE editor --> https://code.earthengine.google.com/  

Once exported, the data can be retrieved on Google Drive and, perhaps, moved to a more appropriate folder.  

Now, let's open the images in QGIS to perform BTD for ash detection, using the following formula:   
>**"B11 - B12 < 0"**

Which is translated in practice by the bands 14 (11µm) and 15 (12µm) of the ABI sensor. to not hesitate to change the threshold value if needed *(e.g., "B11 - B12 = -0.1")*

This will create a mask image for each acquisition, with the same spatial resolution as the GOES-17 images. This mask will be made of pixels have a value of either 1 (respects the formula) or 0 (does not respect the formula). You can change the symbology of the masks to make the "0" value appear transparent, while "1" value can be highlighted by a specific colour.  

<br>

**ASSIGNMENT:**

Take advantage of QGIS to create a stunning layout for each GOES-17 image (10 images). Then, use them as frames to create a GIF animation showing the development of the ash plume over time, together with the BTD ash detection overlaying the satellite images.  

Do not forget the basic rules to make a map (scale and/or coordinates, north arrow, legend, useful content, symbols, etc.)!

There is a python code provided on [GitHub](https://github.com/besmets/RSE_course), in a Jupyter Notebook, to create this GIF animation.  

NOT MANDATORY: You could also make a GIF animation with the TROPOMI imagery (see PART 3), if you would like.  

CONTEST: During March 2024, I will select the best GIF animation(s). The best one will be used to advertise the Master in Geography and the RSE course. So be rigorous and creative!

<br>

***Deadline for submitting the GIF: Monday 26/02/2024, 20:00 (8pm) CET.***

----------------------

## PART 3 – Remote Sensing of trace gases using GEE

In this part, you have to code to retrieve SO2 data from Sentinel-5P TROPOMI.  

Take advantage of what you learned in PART 2!  

You can also have a look at the GEE and GEEMAP documentation:
- https://developers.google.com/earth-engine/datasets
- https://geemap.org/
- https://book.geemap.org/

Finally, you can take inspiration from my JS script to visualise TROPOMI data, [available here on GitHub](https://github.com/besmets/RSE_course/tree/main/Additional_information).